In [5]:
# Importing Dependencies

# For constructing the ConvNetwork that will determine the Q values of moves? 
from keras.models import Sequential
from keras.layers import Dense, Flatten
import numpy as np

# For storing and sampling observations.
import random
from collections import deque

# Construct environment and choose Breakout from OpenAI
import gym
env = gym.make('Breakout-v0')

[2017-08-17 11:33:33,119] Making new env: Breakout-v0


DependencyNotInstalled: No module named 'atari_py'. (HINT: you can install Atari dependencies by running 'pip install gym[atari]'.)

In [ ]:
# God damn atari-py